## Synthetic Chest X-Ray Image Generator 

This solution generates synthetic Chest X-ray images for educational and research purposes by leveraging state of the art generative AI techniques in images. 

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to For Seller to update: Arrhythmia Identification from ECG. 

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

# 1. Subscribe to the model package

To subscribe to the algorithm:
1. Open the model package listing page Synthetic-chest-X-ray-image-generator.
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [28]:
model_package_arn='Synthetic-chest-X-ray-image-generator'

### 2. Prepare dataset

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()
bucket = sagemaker_session.default_bucket()
bucket

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [36]:
model_name = "Synthetic-xray-generator"

content_type = "application/zip"

real_time_inference_instance_type = "ml.g4dn.8xlarge"
batch_transform_inference_instance_type = "ml.p3.2xlarge"

#### A. Create an endpoint

In [37]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session, content_type)

In [38]:
#Create a deployable model from model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn, 
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [39]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

INFO:sagemaker:Creating model with name: Synthetic-chest-X-ray-image-generator-2023-11-03-12-27-44-330
INFO:sagemaker:Creating endpoint-config with name Synthetic-xray-generator
INFO:sagemaker:Creating endpoint with name Synthetic-xray-generator


---------!

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


#### B. Create input payload

#### Instructions

    1) The payload should be in zip format.

In [40]:
file_name = "user_input.zip"

#### C. Perform real-time inference

In [41]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'application/zip' --region us-east-2 output.zip

{
    "ContentType": "application/zip",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- The output file (in zip format) contains the following files:

    1. 'output.zip': A folder containing different synthetic chest X-ray images generated by the model.

In [42]:
import os 
from zipfile import *

file_path = os.getcwd()
file_name_ = 'output.zip'

file_object = open(file_name_,'rb')
z = ZipFile(file_object)
file_names = []
for name in z.namelist():
    z.extract(name,file_path)
    file_names.append(name)
file_object.close()

output_folder_path = os.path.join(file_path, "output")

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [43]:
predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: Synthetic-xray-generator
INFO:sagemaker:Deleting endpoint with name: Synthetic-xray-generator


### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [30]:
#upload the batch-transform job input files to S3
transform_input_folder = file_name
batch_input_prefix = "synthetic-chest-X-ray-generator/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=batch_input_prefix) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/synthetic-chest-X-ray-generator/batch/user_input.zip


In [31]:
#Run the batch inference job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating model with name: Synthetic-chest-X-ray-image-generator-2023-11-03-12-09-04-952
INFO:sagemaker:Creating transform job with name: Synthetic-chest-X-ray-image-generator-2023-11-03-12-09-05-821


..................................................==========
== CUDA ==
CUDA Version 11.7.1
Container image Copyright (c) 2016-2023, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
This container image and its contents are governed by the NVIDIA Deep Learning Container License.
== CUDA ==
CUDA Version 11.7.1
Container image Copyright (c) 2016-2023, NVIDIA CORPORATION & AFFILIATES. All rights reserved.
This container image and its contents are governed by the NVIDIA Deep Learning Container License.
By pulling and using the container, you accept the terms and conditions of this license:
https://developer.nvidia.com/ngc/nvidia-deep-learning-container-license
A copy of this license is made available in this container at /NGC-DL-CONTAINER-LICENSE for your convenience.
2023-11-03 12:17:27.877776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructi

#015Predicting DataLoader 0:  15%|█▌        | 3/20 [01:10<06:42, 23.65s/it]In predict_step module
preprocess reference image (64, 64, 3)
#015Predicting DataLoader 0:  15%|█▌        | 3/20 [01:10<06:42, 23.65s/it]In predict_step module
preprocess reference image (64, 64, 3)
#015Predicting DataLoader 0:  20%|██        | 4/20 [01:34<06:16, 23.53s/it]In predict_step module
preprocess reference image (64, 64, 3)
#015Predicting DataLoader 0:  20%|██        | 4/20 [01:34<06:16, 23.53s/it]In predict_step module
preprocess reference image (64, 64, 3)
#015Predicting DataLoader 0:  25%|██▌       | 5/20 [01:57<05:51, 23.46s/it]In predict_step module
preprocess reference image (64, 64, 3)
#015Predicting DataLoader 0:  25%|██▌       | 5/20 [01:57<05:51, 23.46s/it]In predict_step module
preprocess reference image (64, 64, 3)
preprocess reference image (64, 64, 3)
preprocess reference image (64, 64, 3)
preprocess reference image (64, 64, 3)
preprocess reference image (64, 64, 3)
#015Predicting DataLoa

In [33]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/Synthetic-chest-X-ray-image-generator-2023-11-03-12-09-05-821'

### 4. Clean-up

#### A. Delete the model

In [34]:
model.delete_model()

INFO:sagemaker:Deleting model with name: Synthetic-chest-X-ray-image-generator-2023-11-03-12-09-04-952


#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

